# Preprocesado de datos de la web de hoteles Expedia



## Importamos las librerías necesarias

In [ ]:
import numpy as np 
import pandas as pd

import os

import pandas as pd
import boto3
from io import StringIO
import io
import string
import random
import json
import pickle
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from scipy import stats
from scipy.stats import norm

### Dataset

Datos relativos a 2013-2014

In [ ]:
data = pd.read_csv('train.csv', nrows=100000)

data.head()

In [ ]:
data.columns

In [ ]:
data.info()

#### Variables

|  Variable |  Descripción | 
|----------|:---------|
| date_time  | Timestamp     |
| site_name | ID del punto de venta de Expedia|
| posa_continent | ID del continente del site |
| user_location_country |ID del país del cliente |
| user_location_region|ID de la región del cliente|
| user_location_city| ID of la ciudad del cliente|
| orig_destination_distance| Distancia física entre un hotel y el cliente|
| user_id| ID of usuario|
| is_mobile| 1 para dispositivos móviles, 0 en otro caso|
| is_package| 1 si la reserva es parte de un paquete, 0 en otro caso|
| channel| ID del canal de marketing|
| srch_ci| Fecha de check-in|
| srch_co|Fecha de heck-out|
| srch_adults_cnt| Número de adultos|
| srch_children_cnt| Número de niños|
| srch_rm_cnt| Número de habitaciones|
| srch_destination_id| ID del destino|
| srch_destination_type_id| Tipo de destino|
| is_booking | 1 si se reserva, 0 si se hace click|
| cnt| Número de eventos similares en la misma sesión de usuario|
| hotel_continent| Continente del hotel|
| hotel_country| País del hotely|
| hotel_market| Mercado del hotel|
| hotel_cluster| ID del cluster del hotel|


### Comprobamos el porcentaje de NA que hay en el dataset

In [ ]:

total = data.isnull().sum().sort_values(ascending=False)
percent = (data.isnull().sum()/len(data)).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Porcentaje'])
missing_data.head(20)

### Convertimos a fecha las variables que lo sean de nuestro set de datos:

- date_time
- srch_ci
- srch_co




### Además, generamos nuevas variables a partir de ellas, que sirvan como input a futuros modelos

- stay_dur: duración de la estancia
- no_of_days_bet_booking: número de días entre la reserva y la estancia 
- Cin_day: día del check in
- Cin_month: mes del check in
- Cin_year: año del check-in

In [ ]:
# Function to convert date object into relevant attributes
def convert_date_into_days(df):
    df['srch_ci'] = pd.to_datetime(df['srch_ci'])
    df['srch_co'] = pd.to_datetime(df['srch_co'])
    df['date_time'] = pd.to_datetime(df['date_time'])
    
    df['stay_dur'] = (df['srch_co'] - df['srch_ci']).astype('timedelta64[D]')
    df['no_of_days_bet_booking'] = (df['srch_ci'] - df['date_time']).astype('timedelta64[D]')
    
    df['Cin_day'] = df["srch_ci"].apply(lambda x: x.day)
    df['Cin_month'] = df["srch_ci"].apply(lambda x: x.month)
    df['Cin_year'] = df["srch_ci"].apply(lambda x: x.year)
    return df

In [ ]:
data = convert_date_into_days(data)

In [ ]:
data.info()

In [ ]:
## Número de reservas por año

cin_year = data.Cin_year.value_counts()               
cin_year

In [ ]:
## Número de reservas por mes

cin_month = data.Cin_month.value_counts()               
cin_month

In [ ]:
## Número de reservas por día

cin_days = data.Cin_day.value_counts()    

cin_days

In [ ]:
## Número de reservas por duración de la estancia

stay_dur = data.stay_dur.value_counts()           
stay_dur

### Volvemos a comprobar NA y rellenamos con aquellos valores que son más frecuentes

In [ ]:
total = data.isnull().sum().sort_values(ascending=False)
percent = (data.isnull().sum()/len(data)).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Porcentaje'])
missing_data.head(20)

In [ ]:
data['Cin_day'] = data['Cin_day'].fillna(cin_days.index[0])
data['Cin_month'] = data['Cin_month'].fillna(cin_month.index[0])
data['Cin_year'] = data['Cin_year'].fillna(cin_year.index[0])
data['stay_dur'] = data['stay_dur'].fillna(stay_dur.index[0])
data['no_of_days_bet_booking'] = data['no_of_days_bet_booking'].fillna(0.0)

In [ ]:
# Rellenar los NA en la vriable distancia entre el origen y destino, con la media

data['orig_destination_distance'].fillna(data['orig_destination_distance'].mean(), inplace=True)

In [ ]:
data.head()

### Realmente, tendríamos que haber borrado los registros que no habían acabado en reserva

In [ ]:
data = data[data.is_booking != 0]

### Vemos si hay outliers

In [ ]:
data.columns

In [ ]:
num_cols = ['orig_destination_distance', 'srch_adults_cnt', 'srch_children_cnt',
       'srch_rm_cnt', 'no_of_days_bet_booking', 'stay_dur', 'cnt']
plt.figure(figsize=(18,9))
data[num_cols].boxplot()
plt.title("Variables numéricas", fontsize=20)
plt.show()

#### Vemos que la variable distancia entre origen y destino tiene bastante valores anómalos. 

In [ ]:
data[data.orig_destination_distance > 4000]

#### Además, debido a la diferencia de magnitud con las otras variables, no podemos ver bien si hay outliers en el resto de campos. Vamos a separarlos

In [ ]:
num_cols = ['srch_adults_cnt', 'srch_children_cnt',
       'srch_rm_cnt', 'no_of_days_bet_booking', 'stay_dur', 'cnt']

plt.figure(figsize=(18,9))
data[num_cols].boxplot()
plt.title("Variables numéricas", fontsize=20)
plt.show()

In [ ]:
data[data.no_of_days_bet_booking > 100].orig_destination_distance.mean()

In [ ]:
data[data.no_of_days_bet_booking > 100].orig_destination_distance.median()